![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)
(https://colab.research.google.com/github/ricardokleinklein/NLP_GenMods/blob/main/DALL-E.ipynb)

# Modelos Generativos

## DALL-E - Imagen

Creado por *Ricardo Kleinlein* para [Saturdays.AI](https://saturdays.ai/).

Disponible bajo una licencia [Creative Commons](https://creativecommons.org/licenses/by/4.0/).

---

## Sobre el uso de Jupyter Notebooks

Este notebook ha sido implementado en Python, pero para su ejecución no es
necesario conocer el lenguaje en profundidad. Solamente se debe ejecutar cada
una de las celdas, teniendo en cuenta que hay que ejecutar una celda a la vez
y secuencialmente, tal y como figuran en orden de aparición.

Para ejecutar cada celda pulse en el botón ▶ en la esquina superior izquierda
de cada celda. Mientras se esté ejecutando ese fragmento de código,
el botón estará girando. En caso de querer detener dicha ejecución, pulse
nuevamente sobre este botón mientras gira y la ejecución se detendrá. En caso
de que la celda tenga alguna salida (texto, gráficos, etc) será mostrada
justo después de esta y antes de mostrar la siguiente celda. El notebook
estará guiado con todas las explicaciones necesarias, además irá acompañado
por comentarios en el código para facilitar su lectura.

En caso de tener alguna duda, anótela. Dedicaremos un tiempo a plantear y
resolver la mayoría delas dudas que puedan aparecer.


## Objetivo del notebook

Comprender, implementar y evaluar la generación automática de imágenes a
partir de descripciones textuales usando DALL-E, una red neuronal profunda.

## Importar las librerías necesarias

In [15]:
import jax
import matplotlib.pyplot as plt
from transformers import BartTokenizer

import random
from tqdm.notebook import tqdm
from dalle_mini.model import CustomFlaxBartForConditionalGeneration

from vqgan_jax.modeling_flax_vqgan import VQModel
import numpy as np
from PIL import Image

AttributeError: module 'jax' has no attribute 'numpy'

In [8]:
# Depuración de posibles errores de versiones
DALLE_REPO = 'flax-community/dalle-mini'
DALLE_COMMIT_ID = '4d34126d0df8bc4a692ae933e3b902a1fa8b6114'
#tokenizer = BartTokenizer.from_pretrained(DALLE_REPO,
# revision=DALLE_COMMIT_ID)
model = CustomFlaxBartForConditionalGeneration.from_pretrained(DALLE_REPO, revision=DALLE_COMMIT_ID)

NameError: name 'CustomFlaxBartForConditionalGeneration' is not defined

## Codificación de la descripción

En la celda inferior, vamos a escribir qué queremos que contenga la imagen.
Por ejemplo:

In [ ]:
prompt = 'picture of a dawn in the beach'

### Tokenización

Vamos a procesar las palabras de manera que DALL-E pueda entender qué buscamos.

In [ ]:
tokenized_prompt = tokenizer(prompt, return_tensors='jax',
                             padding='max_length', truncation=True,
                             max_length=128)
print(tokenized_prompt)

### Significado de los tokens

`0`: Representa el inicio de una frase.

`1`: Representa el padding de una secuencia hasta llegar a la longitud máxima.

`2`: Representa el final de una frase.

### Traducción al vocabulario DALL-E

DALL-E es un modelo de investigación, y por tanto su vocabulario está
limitado por los datos de entrenamiento que ha recibido. Por eso, vamos a
traducir a "lenguaje DALL-E" los tokens anteriores.

In [ ]:
n_predictions = 1

# create random keys
seed = random.randint(0, 2**32-1)
key = jax.random.PRNGKey(seed)
subkeys = jax.random.split(key, num=n_predictions)
print(subkeys)

### Obtención de la codificación

In [ ]:
encoded_images = [model.generate(**tokenized_prompt,
                                 do_sample=True, num_beams=1,
                                 prng_key=subkey) for subkey in tqdm(subkeys)]
print(encoded_images[0])
# remove first token (BOS)
encoded_images = [img.sequences[..., 1:] for img in encoded_images]
print(encoded_images[0])

En este momento tenemos posibles imágenes que representan lo que deseábamos,
 pero todas ellas están codificadas por un vector de 256 dimensiones.

In [ ]:
print(encoded_images[0].shape)

## Decodificación & Generación

Primero tenemos que descargar/cargar el modelo pre-entrenado:

In [ ]:
# make sure we use compatible versions
VQGAN_REPO = 'flax-community/vqgan_f16_16384'
VQGAN_COMMIT_ID = '90cc46addd2dd8f5be21586a9a23e1b95aa506a9'
vqgan = VQModel.from_pretrained(VQGAN_REPO, revision=VQGAN_COMMIT_ID)

### Generar!

In [ ]:
decoded_images = [vqgan.decode_code(encoded_image) for encoded_image in tqdm(encoded_images)]
print(decoded_images[0])

### Últimos ajustes para visualizar

Las imágenes decodificadas no pueden ser visualizadas tal cual están ahora.
Primero debemos formatearlas de acuerdo con los protocolos en los que fue
diseñado el modelo originalmente.

In [ ]:
clipped_images = [img.squeeze().clip(0., 1.) for img in decoded_images]
images = [Image.fromarray(np.asarray(img * 255, dtype=np.uint8)) for img in clipped_images]

print(images[0])
plt.imshow(images[0])
